In [15]:
import numpy as np 
import pandas as pd
import math
from sklearn.tree import DecisionTreeClassifier
np.seterr(divide = 'ignore')

{'divide': 'ignore', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [16]:
cardio_train_df = pd.read_csv('/content/cardio_train.csv', sep= ';')
cardio_train_df

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,99993,19240,2,168,76.0,120,80,1,1,1,0,1,0
69996,99995,22601,1,158,126.0,140,90,2,2,0,0,1,1
69997,99996,19066,2,183,105.0,180,90,3,1,0,1,0,1
69998,99998,22431,1,163,72.0,135,80,1,2,0,0,0,1


In [20]:
# calculate the total entropy for training examples
# def calc_total_entropy(input_dataframe, target_prediction, target_unique_values):
#   total_rows = input_dataframe.shape[0]
#   total_entropy = 0
#   for target_unique_value in target_unique_values:
#     class_freq = input_dataframe[input_dataframe[target_prediction]==target_unique_value].shape[0]
#     total_class_entr = - (class_freq/total_rows)*np.log2(class_freq/total_rows) #entropy of the class
#     total_entropy += total_class_entr 
#   return total_entropy

In [17]:
def calc_total_entropy(train_data, label, class_list):
    total_row = train_data.shape[0] #the total size of the dataset
    total_entr = 0
    
    for c in class_list: #for each class in the label
        total_class_count = train_data[train_data[label] == c].shape[0] #number of the class
        total_class_entr = - (total_class_count/total_row)*np.log2(total_class_count/total_row) #entropy of the class
        total_entr += total_class_entr #adding the class entropy to the total entropy of the dataset
    
    return total_entr

In [4]:
#calc entropy for a specfic feature unique value
# def calc_entropy(feature_value_dataframe, target_prediction, target_unique_values):
#     total_rows = feature_value_dataframe.shape[0]
#     entropy = 0
#     for target_unique_value in target_unique_values:
#         label_class_count = feature_value_dataframe[feature_value_dataframe[target_prediction] == target_unique_value].shape[0] 
#         entropy_class = 0
#         if label_class_count != 0:
#             probability_class = label_class_count/total_rows 
#             entropy_class = - probability_class * np.log2(probability_class) 
#         entropy += entropy_class
#     return entropy

In [18]:
def calc_entropy(feature_value_data, label, class_list):
    class_count = feature_value_data.shape[0]
    entropy = 0
    
    for c in class_list:
        label_class_count = feature_value_data[feature_value_data[label] == c].shape[0] #row count of class c 
        entropy_class = 0
        if label_class_count != 0:
            probability_class = label_class_count/class_count #probability of the class
            entropy_class = - probability_class * np.log2(probability_class)  #entropy
        entropy += entropy_class
    return entropy

In [5]:
#the function calculates the information gain for a feature
# def calc_information_gain(input_dataframe, feature_name, target_prediction, target_unique_values):
#   total_rows = input_dataframe.shape[0]
#   total_entropy = calc_total_entropy(input_dataframe, target_prediction, target_unique_values)
#   unique_feature_values = np.unique(input_dataframe[feature_name], return_counts = True)[0]
#   feature_value_entropy = []
#   feature_value_probability = []
#   for unique_feature_value in unique_feature_values:
#     feature_value_dataframe = input_dataframe[input_dataframe[feature_name] == unique_feature_value]
#     feature_value_entropy.append(calc_entropy(feature_value_dataframe, target_prediction, target_unique_values))
#     feature_value_probability.append(feature_value_dataframe.shape[0]/total_rows)
#   feature_entopy = 0
#   for i in range(len(feature_value_entropy)):
#     feature_entopy += -feature_value_entropy[i]* feature_value_probability[i]
#   total_gain = total_entropy + feature_entopy
#   return total_gain

In [19]:
def calc_info_gain(feature_name, train_data, label, class_list):
    feature_value_list = train_data[feature_name].unique() #unqiue values of the feature
    total_row = train_data.shape[0]
    feature_info = 0.0
    
    for feature_value in feature_value_list:
        feature_value_data = train_data[train_data[feature_name] == feature_value] #filtering rows with that feature_value
        feature_value_count = feature_value_data.shape[0]
        feature_value_entropy = calc_entropy(feature_value_data, label, class_list) #calculcating entropy for the feature value
        feature_value_probability = feature_value_count/total_row
        feature_info += feature_value_probability * feature_value_entropy #calculating information of the feature value
        
    return calc_total_entropy(train_data, label, class_list) - feature_info #calculating information gain by subtracting

In [20]:
def find_most_informative_feature(train_data, label, class_list):
    feature_list = train_data.columns.drop(label) #finding the feature names in the dataset
                                            #N.B. label is not a feature, so dropping it
    max_info_gain = -1
    max_info_feature = None
    
    for feature in feature_list:  #for each feature in the dataset
        feature_info_gain = calc_info_gain(feature, train_data, label, class_list)
        if max_info_gain < feature_info_gain: #selecting feature name with highest information gain
            max_info_gain = feature_info_gain
            max_info_feature = feature
            
    return max_info_feature

In [6]:
# def highest_info_gain_feature(input_dataframe, target_prediction, target_unique_values):
#   feature_list = input_dataframe.columns.drop(target_prediction)                              
#   max_info_gain = -1
#   max_info_feature = None
#   for feature in feature_list:  #for each feature in the dataset
#         feature_info_gain = calc_information_gain(input_dataframe, feature, target_prediction, target_unique_values)
#         if max_info_gain < feature_info_gain: 
#             max_info_gain = feature_info_gain
#             max_info_feature = feature
#   return max_info_feature

In [21]:
def find_most_informative_feature(train_data, label, class_list):
    feature_list = train_data.columns.drop(label) #finding the feature names in the dataset
                                            #N.B. label is not a feature, so dropping it
    max_info_gain = -1
    max_info_feature = None
    
    for feature in feature_list:  #for each feature in the dataset
        feature_info_gain = calc_info_gain(feature, train_data, label, class_list)
        if max_info_gain < feature_info_gain: #selecting feature name with highest information gain
            max_info_gain = feature_info_gain
            max_info_feature = feature
            
    return max_info_feature

In [22]:
def generate_sub_tree(feature_name, train_data, label, class_list):
    feature_value_count_dict = train_data[feature_name].value_counts(sort=False) #dictionary of the count of unqiue feature value
    tree = {} #sub tree or node
    
    for feature_value, count in feature_value_count_dict.iteritems():
        feature_value_data = train_data[train_data[feature_name] == feature_value] #dataset with only feature_name = feature_value
        
        assigned_to_node = False #flag for tracking feature_value is pure class or not
        for c in class_list: #for each class
            class_count = feature_value_data[feature_value_data[label] == c].shape[0] #count of class c

            if class_count == count: #count of (feature_value = count) of class (pure class)
                tree[feature_value] = c #adding node to the tree
                train_data = train_data[train_data[feature_name] != feature_value] #removing rows with feature_value
                assigned_to_node = True
        if not assigned_to_node: #not pure class
            tree[feature_value] = "?" #as feature_value is not a pure class, it should be expanded further, 
                                      #so the branch is marking with ?
            
    return tree, train_data

In [23]:
def make_tree(root, prev_feature_value, train_data, label, class_list):
    if train_data.shape[0] != 0: #if dataset becomes enpty after updating
        max_info_feature = find_most_informative_feature(train_data, label, class_list) #most informative feature
        tree, train_data = generate_sub_tree(max_info_feature, train_data, label, class_list) #getting tree node and updated dataset
        next_root = None
        
        if prev_feature_value != None: #add to intermediate node of the tree
            root[prev_feature_value] = dict()
            root[prev_feature_value][max_info_feature] = tree
            next_root = root[prev_feature_value][max_info_feature]
        else: #add to root of the tree
            root[max_info_feature] = tree
            next_root = root[max_info_feature]
        
        for node, branch in list(next_root.items()): #iterating the tree node
            if branch == "?": #if it is expandable
                feature_value_data = train_data[train_data[max_info_feature] == node] #using the updated dataset
                make_tree(next_root, node, feature_value_data, label, class_list) #recursive call with updated dataset


In [24]:
def id3(train_data_m, label):
    train_data = train_data_m.copy() #getting a copy of the dataset
    tree = {} #tree which will be updated
    class_list = train_data[label].unique() #getting unqiue classes of the label
    make_tree(tree, None, train_data, label, class_list) #start calling recursion
    return tree

In [25]:
def predict(tree, instance):
    if not isinstance(tree, dict): #if it is leaf node
        return tree #return the value
    else:
        root_node = next(iter(tree)) #getting first key/feature name of the dictionary
        feature_value = instance[root_node] #value of the feature
        if feature_value in tree[root_node]: #checking the feature value in current tree node
            return predict(tree[root_node][feature_value], instance) #goto next feature
        else:
            return None

In [26]:
def evaluate(tree, test_data_m, label):
    correct_preditct = 0
    wrong_preditct = 0
    for index, row in test_data_m.iterrows(): #for each row in the dataset
        result = predict(tree, test_data_m.iloc[index]) #predict the row
        if result == test_data_m[label].iloc[index]: #predicted value and expected value is same or not
            correct_preditct += 1 #increase correct count
        else:
            wrong_preditct += 1 #increase incorrect count
    accuracy = correct_preditct / (correct_preditct + wrong_preditct) #calculating accuracy
    return accuracy

In [27]:
tree = id3(cardio_train_df, 'cardio')

In [29]:
accuracy = evaluate(tree, cardio_train_df, 'cardio')
accuracy

1.0

In [30]:
from sklearn.model_selection import cross_val_score
clf = DecisionTreeClassifier(criterion="entropy")
np.mean(cross_val_score(clf, cardio_train_df, cardio_train_df['cardio'], cv=10))

1.0